# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
 # Data Science
import pandas as pd
import numpy as np

# API Requests
from pprint import pprint
import requests
import json

# Data Viz
import hvplot.pandas
import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys_prof import geoapify_key

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,allapalli,19.4167,80.0667,55.74,49,97,2.77,IN,1734047475
1,utrik,11.2278,169.8474,81.72,79,29,22.50,MH,1734047477
2,chonchi,-42.6128,-73.8092,59.13,81,66,4.16,CL,1734047478
3,port-aux-francais,-49.3500,70.2167,41.04,83,83,30.53,TF,1734047480
4,soavinandriana,-19.1667,46.7333,60.28,89,6,5.23,MG,1734047481


In [18]:
city_data_df["Date"] = pd.to_datetime(city_data_df.Date * 1e9)
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,allapalli,19.4167,80.0667,55.74,49,97,2.77,IN,2024-12-12 23:51:15
1,utrik,11.2278,169.8474,81.72,79,29,22.50,MH,2024-12-12 23:51:17
2,chonchi,-42.6128,-73.8092,59.13,81,66,4.16,CL,2024-12-12 23:51:18
3,port-aux-francais,-49.3500,70.2167,41.04,83,83,30.53,TF,2024-12-12 23:51:20
4,soavinandriana,-19.1667,46.7333,60.28,89,6,5.23,MG,2024-12-12 23:51:21


In [21]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 572 entries, 0 to 571
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   City        572 non-null    object        
 1   Lat         572 non-null    float64       
 2   Lng         572 non-null    float64       
 3   Max Temp    572 non-null    float64       
 4   Humidity    572 non-null    int64         
 5   Cloudiness  572 non-null    int64         
 6   Wind Speed  572 non-null    float64       
 7   Country     569 non-null    object        
 8   Date        572 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 44.7+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    color = "City",
    hover_cols=["City", "Country", "Max Temp"]  
)


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [34]:
# Narrow down cities that fit criteria and drop any results with null values
Min_temp = 73
Max_temp = 84
Max_wind = 16

# Drop any rows with null values
df_cleaned = city_data_df.dropna()
city_data_df.dropna(inplace=True)
city_data_df.dropna(subset=['Lat', 'Max Temp'])


# Display sample data
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,allapalli,19.4167,80.0667,55.74,49,97,2.77,IN,2024-12-12 23:51:15
1,utrik,11.2278,169.8474,81.72,79,29,22.50,MH,2024-12-12 23:51:17
2,chonchi,-42.6128,-73.8092,59.13,81,66,4.16,CL,2024-12-12 23:51:18
3,port-aux-francais,-49.3500,70.2167,41.04,83,83,30.53,TF,2024-12-12 23:51:20
4,soavinandriana,-19.1667,46.7333,60.28,89,6,5.23,MG,2024-12-12 23:51:21
...,...,...,...,...,...,...,...,...,...
567,santiago,-33.4569,-70.6483,72.86,51,0,19.57,CL,2024-12-12 23:57:55
568,canyon,34.9803,-101.9188,50.67,33,100,8.01,US,2024-12-13 00:02:51
569,camapua,-19.5314,-54.0439,76.08,85,100,1.23,BR,2024-12-13 00:02:53


### Step 3: Create a new DataFrame called `hotel_df`.

In [37]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
0,allapalli,IN,19.4167,80.0667,49,
1,utrik,MH,11.2278,169.8474,79,
2,chonchi,CL,-42.6128,-73.8092,81,
3,port-aux-francais,TF,-49.3500,70.2167,83,
4,soavinandriana,MG,-19.1667,46.7333,89,
...,...,...,...,...,...,...
567,santiago,CL,-33.4569,-70.6483,51,
568,canyon,US,34.9803,-101.9188,33,
569,camapua,BR,-19.5314,-54.0439,85,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [43]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":17
}

    
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
allapalli - nearest hotel: No hotel found
utrik - nearest hotel: No hotel found
chonchi - nearest hotel: Hotel Huildín
port-aux-francais - nearest hotel: Keravel
soavinandriana - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
adamstown - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
iqaluit - nearest hotel: Frobisher Inn
qamdo - nearest hotel: 银杏商务宾馆
nieuw amsterdam - nearest hotel: Leonsberg Lodge
nyrob - nearest hotel: No hotel found
west island - nearest hotel: Cocos Village Bungalows
sao vicente - nearest hotel: Pousada Vitória
klyuchi - nearest hotel: No hotel found
puerto natales - nearest hotel: Dorotea Patagonia Hostel
albany - nearest hotel: No hotel found
margate - nearest hotel: The Reading Rooms
bethel - nearest hotel: Hampton Inn Danbury
yellowknife - nearest hotel: No hotel found
mugia - nearest hotel: Solpor do Coído
jamestown - nearest hotel: DoubleTree Jamestown
bilibino - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
0,allapalli,IN,19.4167,80.0667,49,No hotel found
1,utrik,MH,11.2278,169.8474,79,No hotel found
2,chonchi,CL,-42.6128,-73.8092,81,Hotel Huildín
3,port-aux-francais,TF,-49.3500,70.2167,83,Keravel
4,soavinandriana,MG,-19.1667,46.7333,89,No hotel found
...,...,...,...,...,...,...
567,santiago,CL,-33.4569,-70.6483,51,apart arturo prat
568,canyon,US,34.9803,-101.9188,33,Buffalo Inn
569,camapua,BR,-19.5314,-54.0439,85,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.75,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)